In [ ]:
# -----------------------------------------------------------------------------
# 📘 Notebook: 02_data_cleaning_and_summary.ipynb
#
# Purpose:
#   Prepare a clean, high-quality summary dataset of all running sessions.
#   This notebook filters out non-running or corrupted data, aggregates
#   individual GPS records into one-row-per-run summaries, and enriches them
#   with temporal and quality information.
#
# Context:
#   - Follows Stage 1 (EDA & Data Quality Audit)
#   - Produces a ready-to-ingest dataset for Neo4j and ML analysis
#
# Main steps:
#   1. Load raw processed data (strava_runs.parquet)
#   2. Apply cleaning and filtering rules based on the data quality manifest
#   3. Aggregate per run (distance, pace, cadence, heart rate, elevation)
#   4. Add date/time and quality metrics
#   5. Save the final summary to:
#        data/strava/processed/run_summary_cleaned.{csv,parquet}
#
# Input:
#   data/strava/processed/strava_runs.parquet
#   data/strava/processed/data_quality_manifest.csv
#
# Output:
#   data/strava/processed/run_summary_cleaned.parquet
#   data/strava/processed/run_summary_cleaned.csv
#
# Next step:
#   Stage 3 → Neo4j graph ingestion and visualization
# -----------------------------------------------------------------------------


In [ ]:
# --- Stage 2: Cleaning & Per-Run Summary ---------------------------------

import pandas as pd
import numpy as np
from pathlib import Path

raw_path = Path("../data/strava/processed/strava_runs.parquet")
df = pd.read_parquet(raw_path)
print(f"Loaded {len(df):,} rows")


In [ ]:
# --- 2.0 Create or refresh data quality manifest --------------------------
from pathlib import Path
import pandas as pd

processed_dir = Path("data/strava/processed")
manifest_path = processed_dir / "data_quality_manifest.csv"

# Whether to force regeneration
force_recreate_manifest = True
processed_dir.mkdir(parents=True, exist_ok=True)

def classify_column(col: str, missing_pct: float, nunique: int) -> str:
    """Heuristic classification of columns."""
    col_lower = col.lower()
    if col_lower.startswith("unknown") or "unnamed" in col_lower:
        return "drop"
    if missing_pct > 70:
        return "review"
    if missing_pct > 50:
        return "optional"
    if nunique <= 1:
        return "drop"  # constant or empty
    if col_lower in [
        "distance", "distance_km", "pace_min_per_km", "timestamp",
        "speed", "cadence", "altitude", "heart_rate", "run_id",
        "lat", "lon", "position_lat", "position_long",
    ]:
        return "core"
    return "optional"

if not manifest_path.exists() or force_recreate_manifest:
    print("🔄 Creating or refreshing data quality manifest...")

    # Compute stats
    stats = pd.DataFrame({
        "column": df.columns,
        "dtype": [str(df[c].dtype) for c in df.columns],
        "non_null_count": df.notna().sum().values,
        "missing_pct": (df.isna().mean().values * 100).round(1),
        "nunique": df.nunique().values,
    })

    stats["category"] = [
        classify_column(c, m, n)
        for c, m, n in zip(stats["column"], stats["missing_pct"], stats["nunique"])
    ]

    stats = stats.sort_values(["category", "missing_pct"], ascending=[True, False])

    stats.to_csv(manifest_path, index=False)
    print(f"✅ Manifest refreshed with {len(stats)} columns at: {manifest_path.resolve()}")
    print(stats["category"].value_counts().to_string())
else:
    print(f"✅ Manifest already exists and not overwritten: {manifest_path.resolve()}")

# Preview top of manifest
pd.read_csv(manifest_path).head(10)


In [ ]:
# --- 2.1 Drop unknown or low-quality columns ------------------------------
manifest = pd.read_csv("../data/strava/processed/data_quality_manifest.csv", index_col="column")
keep_cols = manifest.query("category != 'drop'").index.tolist()
df = df[keep_cols]
print(f"Keeping {len(keep_cols)} columns based on manifest")


In [ ]:
# --- 2.2 Apply activity-level filters ------------------------------------
before = len(df)
df = df.dropna(subset=["distance_km", "pace_min_per_km"])
df = df[df["distance_km"] >= 0.5]
df = df[(df["pace_min_per_km"] >= 2) & (df["pace_min_per_km"] <= 15)]
df = df[~((df["cadence"] == 0) & (df["speed"] == 0))]
print(f"Removed {before - len(df):,} noisy rows")


In [ ]:
# --- 2.3 Aggregate per-run summary (robust to missing HR) ----------------
agg_spec = dict(
    records=("timestamp", "count"),
    start_time=("timestamp", "min"),
    end_time=("timestamp", "max"),
    total_distance_km=("distance_km", "max"),
    avg_pace=("pace_min_per_km", "mean"),
    avg_speed=("speed", "mean"),
    avg_cadence=("cadence", "mean"),
    elevation_gain=("altitude", lambda s: s.max() - s.min()),
)

if "heart_rate" in df.columns:
    agg_spec["avg_hr"] = ("heart_rate", "mean")

summary = df.groupby("run_id").agg(**agg_spec).reset_index()


In [ ]:
# --- 2.4 Add date context -------------------------------------------------
summary["date"] = pd.to_datetime(summary["start_time"]).dt.date
summary["weekday"] = pd.to_datetime(summary["start_time"]).dt.day_name()
summary["month"] = pd.to_datetime(summary["start_time"]).dt.to_period("M")



In [ ]:
# --- 2.5 Run-level quality metrics ---------------------------------------

# per-run % missing values
missing_pct = (
    df.isna()
      .groupby(df["run_id"])
      .mean()
      .mean(axis=1) * 100
)
summary = summary.merge(missing_pct.rename("missing_pct"), on="run_id", how="left")

# per-run duration in minutes
summary["duration_min"] = (
    (pd.to_datetime(summary["end_time"]) - pd.to_datetime(summary["start_time"]))
    .dt.total_seconds() / 60
)


In [ ]:
# --- 2.6 Save results -----------------------------------------------------
out_path = Path("../data/strava/processed/run_summary_cleaned.parquet")
summary.to_parquet(out_path, index=False)
summary.to_csv(out_path.with_suffix(".csv"), index=False)
print(f"✅ Saved {len(summary):,} clean runs → {out_path}")

summary.head()